In [1]:
!pip install selenium
!pip install webdriver-manager

In [87]:
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
from tqdm import tqdm
import numpy as np

In [88]:
df = pd.read_excel("1970_2019_영화.xlsx")
names = df.movieNm
years = df.prdtYear

In [89]:
addr = "https://movie.naver.com/movie/bi/mi/basic.nhn?code=149236" 

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(addr)

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [C:\Users\beomj\.wdm\drivers\chromedriver\win32\84.0.4147.30\chromedriver.exe] found in cache
 


In [90]:
def find(name, year):
    stories = []
    rates = []
    reviews = []
    try:
        temp = driver.find_element_by_class_name("ipt_tx_srch")
        temp.send_keys(name)
        driver.find_element_by_class_name("btn_srch").click()
        time.sleep(1)
        temp = driver.find_element_by_class_name("search_list_1")
        time.sleep(1)
        temp = temp.find_elements_by_tag_name("li")
        found =False
        for i in temp:
            if year in i.text:
                found =True
                temp = i.find_element_by_tag_name("dt")
                temp = temp.find_element_by_tag_name("a").click()
                temp = driver.find_element_by_class_name("story_area")
                break
        if not found:
            for i in temp:
                if str(int(year)-1) in i.text or str(int(year)+1) in i.text :
                    found =True
                    temp = i.find_element_by_tag_name("dt")
                    temp = temp.find_element_by_tag_name("a").click()
                    temp = driver.find_element_by_class_name("story_area")
                    break
        stories.append(temp.text)

        try:
            temp = driver.find_element_by_id("movieEndTabMenu")
            temp.find_element_by_link_text("평점").click()
            time.sleep(0.5)
            temp = driver.find_element_by_id("netizen_point_tab_inner")
            rates.append(temp.text)
        except:
            rates.append("")    
        
        try:
            total = []
            for i in range(1, 50):
                driver.switch_to_default_content()
                driver.switch_to_frame('pointAfterListIframe')
                try:
                    if i != 1:
                        driver.find_element_by_id("pagerTagAnchor"+str(i)).click()
                        time.sleep(0.3)
                except:
                    break
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')

                content_list = soup.find("div", class_ = "score_result").find_all('li')
                try:
                    for li in content_list :
                        temp_review = li.find("div", class_ = "score_reple").find("p").get_text()
                        total.append(temp_review.strip())
                except:
                    pass
            total = "///".join(total)
            total = total.replace("\n", "")
            total = total.replace("\t", "")
            total = total.replace("관람객", "")
            total = total.replace("스포일러가 포함된 감상평입니다. 감상평 보기", "")
            reviews.append(total)
        except:
            reviews.append("")
    except:
        stories.append("")
        rates.append("")
        reviews.append("")
        
        addr = "https://movie.naver.com/movie/bi/mi/basic.nhn?code=149236" 
        driver.get(addr)
    addr = "https://movie.naver.com/movie/bi/mi/basic.nhn?code=149236" 
    driver.get(addr)
    return stories, rates, reviews

In [98]:
data = pd.DataFrame({'영화명':names})
data['스토리'] = None
data['평점'] = None
data['리뷰'] = None
data.head()

,영화명,스토리,평점,리뷰
0,지니어스 독,None,None,None
1,뉴 뮤턴트,None,None,None
2,드라이브,None,None,None
3,쥬라기 썬더,None,None,None
4,바다 몬스터2,None,None,None


In [99]:
start = 4000
end = len(df)
for i in tqdm(range(start, end)):
    if data.loc[i, '스토리'] is None:
        try:
            year = str(int(years[i]))
        except:
            year = 0
        name = names[i]
        s,r,re = find(name, year)
        data.loc[i,'스토리'] =s[0]  
        data.loc[i, '평점']  =r[0]  
        data.loc[i, '리뷰']  =re[0]  

100%|██████████| 1825/1825 [7:39:08<00:00, 15.10s/it]


In [100]:
data.to_excel("1970_2019_naver"+str(start)+"_"+str(end)+".xlsx", index=False)

In [109]:
data[4000:]

,영화명,스토리,평점,리뷰
4000,블라이가의 미로,줄거리\n대학을 졸업하고 직장을 찾던 제니(Jenny: 팻시 켄싯 분)는 쿠퍼(Mr...,,
4001,노 임팩트 맨,줄거리\n작가이자 환경운동가인 ‘콜린’은 1년간 가족과 함께 지구에 무해(無害)한 ...,8.72,노 임팩트맨 책을 읽고 보는데 저희에게 큰 임팩트를 주는 것 같아요. 정치인을 좋은...
4002,오페라- 햄릿 (메트로폴리탄),,,
4003,A-특공대,줄거리\n최고의 실력을 자랑하던 특공대가 돌연 자취를 감춘 지 1년. 누구도 해결할...,8.93,이도 저도 아닌 어중간한 영화를 만들바엔 차라리 이렇게 만들어야한다///기승전결이 ...
4004,섹스 앤 더 시티 2,줄거리\n더 화려하게 더 당당하게\n캐리(사라 제시카 파커)가 오랜 연인이었던 빅과...,7.75,된장녀라 사치녀라 불려도 내겐 부럽고 멋있는 여성들의 스토리를 담은 최고의 드라마/...
...,...,...,...,...
5820,한나스 워,줄거리\n1921년 부다페스트에서 저명한 극작가의 딸로 태어난 한나(Hanna: 마...,9.39,좀 오래된 영화이지만 명작하라 하기엔 충분하다///내가 본 최고의 명화이지만 찾아보...
5821,핫스 오브 화이어,,,
5822,핫타켓,,,
5823,햄버거 힐,줄거리\n{제101공수부대(Troops Of The 101st Airborne Di...,,
